In [1]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

class ResNet34(nn.Module):
    def __init__(self, num_classes=102):
        super(ResNet34, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(BasicBlock, 64, 3)
        self.layer2 = self._make_layer(BasicBlock, 128, 4, stride=2)
        self.layer3 = self._make_layer(BasicBlock, 256, 6, stride=2)
        self.layer4 = self._make_layer(BasicBlock, 512, 3, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


In [2]:
import os
import scipy.io
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class FlowerDataset(Dataset):
    def __init__(self, image_dir, labels, indices, transform=None):
        self.image_dir = image_dir
        self.labels = labels
        self.indices = indices
        self.transform = transform
        self.images = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        img_idx = self.indices[idx]
        img_path = os.path.join(self.image_dir, self.images[img_idx - 1])  # Adjust index for 1-based MATLAB indexing
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels[img_idx - 1] - 1  # Adjust labels to 0-based indexing
        return image, label

# Load dataset files
image_dir = "./jpg"
labels_mat = scipy.io.loadmat("imagelabels.mat")['labels'][0]
split_mat = scipy.io.loadmat("setid.mat")

# Get splits
train_indices = split_mat['trnid'][0]
valid_indices = split_mat['valid'][0]
test_indices = split_mat['tstid'][0]

# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = FlowerDataset(image_dir, labels_mat, train_indices, transform=train_transform)
valid_dataset = FlowerDataset(image_dir, labels_mat, valid_indices, transform=test_transform)
test_dataset = FlowerDataset(image_dir, labels_mat, test_indices, transform=test_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm



# Initialize model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet34(num_classes=102).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Training loop
def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print('-' * 30)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(valid_loader.dataset)
        val_acc = val_corrects.double() / len(valid_loader.dataset)
        print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # Save the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_resnet34.pth')

        # Step the scheduler
        scheduler.step()

    print(f"Best Val Acc: {best_val_acc:.4f}")

# Train the model
train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=5)


Epoch 1/5
------------------------------


100%|██████████| 16/16 [01:43<00:00,  6.44s/it]


Train Loss: 7.1923 Acc: 0.0020
Val Loss: 7109.2048 Acc: 0.0127
Epoch 2/5
------------------------------


100%|██████████| 16/16 [02:20<00:00,  8.79s/it]


Train Loss: 4.7133 Acc: 0.0157
Val Loss: 9.2113 Acc: 0.0196
Epoch 3/5
------------------------------


...

Train Loss: 4.4679 Acc: 0.0157
Val Loss: 4.4634 Acc: 0.0304
Epoch 4/5
------------------------------


100%|██████████| 16/16 [02:20<00:00,  8.81s/it]


Train Loss: 4.3435 Acc: 0.0284
Val Loss: 4.3290 Acc: 0.0304
Epoch 5/5
------------------------------


100%|██████████| 16/16 [02:25<00:00,  9.10s/it]


Train Loss: 4.1984 Acc: 0.0333
Val Loss: 4.1493 Acc: 0.0431
Best Val Acc: 0.0431


In [4]:
def predict(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in tqdm(test_loader):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
    return predictions

# Load the best model
model.load_state_dict(torch.load('best_resnet34.pth'))

# Make predictions on the test set
test_predictions = predict(model, test_loader)
print("Predictions on Test Set:", test_predictions)


/var/folders/wk/3h_3p7yj55q5prg62hx26fx00000gn/T/ipykernel_6036/2834191714.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_resnet

Predictions on Test Set: [90, 36, 36, 36, 36, 36, 36, 36, 36, 57, 36, 78, 36, 36, 90, 36, 36, 36, 36, 90, 36, 36, 90, 78, 36, 90, 90, 36, 90, 36, 36, 36, 36, 36, 78, 90, 36, 57, 90, 36, 86, 36, 36, 7, 36, 78, 90, 36, 36, 36, 36, 36, 36, 78, 36, 36, 36, 36, 57, 36, 36, 36, 90, 36, 36, 84, 57, 90, 36, 36, 84, 90, 90, 36, 36, 90, 57, 36, 36, 36, 57, 57, 36, 90, 36, 90, 90, 36, 90, 84, 90, 36, 36, 55, 36, 36, 90, 90, 36, 36, 55, 57, 36, 36, 36, 90, 91, 84, 36, 36, 36, 36, 57, 36, 55, 90, 70, 70, 70, 70, 70, 70, 70, 70, 14, 70, 70, 14, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 14, 70, 70, 70, 70, 70, 70, 70, 70, 14, 70, 70, 70, 70, 14, 70, 70, 70, 86, 57, 14, 7, 14, 57, 78, 86, 90, 14, 7, 7, 70, 59, 7, 7, 78, 7, 78, 86, 14, 57, 78, 57, 7, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 7, 7, 36, 7, 14, 14, 7, 7, 78, 78, 7, 7, 7, 7, 78, 7, 7, 7, 7, 78, 78, 36, 7, 78, 86, 7, 7, 7, 7, 7, 7, 7, 7, 78, 7, 78, 7, 42, 7, 7, 86, 14, 78, 7, 36, 7